# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f154cecc3a0>
├── 'a' --> tensor([[ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290]])
└── 'x' --> <FastTreeValue 0x7f1548debfa0>
    └── 'c' --> tensor([[ 1.0546,  0.7291,  0.1206, -0.0410],
                        [-2.1992, -0.1414,  1.2098, -0.0544],
                        [ 0.4711,  0.0173,  0.8568, -0.4992]])

In [4]:
t.a

tensor([[ 0.1496, -1.2800,  0.8527],
        [-1.6374, -0.0340, -0.0290]])

In [5]:
%timeit t.a

60.5 ns ± 0.041 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f154cecc3a0>
├── 'a' --> tensor([[-0.2510,  0.1676,  2.6542],
│                   [ 1.7389, -0.9621, -1.3552]])
└── 'x' --> <FastTreeValue 0x7f1548debfa0>
    └── 'c' --> tensor([[ 1.0546,  0.7291,  0.1206, -0.0410],
                        [-2.1992, -0.1414,  1.2098, -0.0544],
                        [ 0.4711,  0.0173,  0.8568, -0.4992]])

In [7]:
%timeit t.a = new_value

59.4 ns ± 0.0383 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290]]),
    x: Batch(
           c: tensor([[ 1.0546,  0.7291,  0.1206, -0.0410],
                      [-2.1992, -0.1414,  1.2098, -0.0544],
                      [ 0.4711,  0.0173,  0.8568, -0.4992]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1496, -1.2800,  0.8527],
        [-1.6374, -0.0340, -0.0290]])

In [11]:
%timeit b.a

51.5 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7790,  1.2962,  1.2477],
               [-0.5858, -1.3944,  0.8464]]),
    x: Batch(
           c: tensor([[ 1.0546,  0.7291,  0.1206, -0.0410],
                      [-2.1992, -0.1414,  1.2098, -0.0544],
                      [ 0.4711,  0.0173,  0.8568, -0.4992]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.0898 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

869 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 62.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 934 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 319 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f149dadfac0>
├── 'a' --> tensor([[[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]],
│           
│                   [[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]],
│           
│                   [[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]],
│           
│                   [[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]],
│           
│                   [[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]],
│           
│                   [[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]],
│           
│                   [[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]],
│           
│                   [[ 0.1496, -1.2800,  0.8527],
│                    [-1.6374, -0.0340, -0.0290]]])
└── 'x' --> <FastTreeValue 0x7f149dadff10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 80.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f149dbd75b0>
├── 'a' --> tensor([[ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290],
│                   [ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290],
│                   [ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290],
│                   [ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290],
│                   [ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290],
│                   [ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290],
│                   [ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290],
│                   [ 0.1496, -1.2800,  0.8527],
│                   [-1.6374, -0.0340, -0.0290]])
└── 'x' --> <FastTreeValue 0x7f14a49bad90>
    └── 'c' --> tensor([[ 1.0546,  0.7291,  0.1206, -0.0410],
                        [-2.1992, -0.1414,  1.2098, -0.0544],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.2 µs ± 56.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]],
       
               [[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]],
       
               [[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]],
       
               [[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]],
       
               [[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]],
       
               [[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]],
       
               [[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]],
       
               [[ 0.1496, -1.2800,  0.8527],
                [-1.6374, -0.0340, -0.0290]]]),
    x: Batch(
           c: tensor([[[ 1.0546,  0.7291,  0.1206, -0.0410],
                       [-2.1992, -0.1414,  1.2098, -0.0544],
                       [ 0.4711,  0.0173,  0.8568, -0.4992]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290],
               [ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290],
               [ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290],
               [ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290],
               [ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290],
               [ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290],
               [ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290],
               [ 0.1496, -1.2800,  0.8527],
               [-1.6374, -0.0340, -0.0290]]),
    x: Batch(
           c: tensor([[ 1.0546,  0.7291,  0.1206, -0.0410],
                      [-2.1992, -0.1414,  1.2098, -0.0544],
                      [ 0.4711,  0.0173,  0.8568, -0.4992],
                      [ 1.0546,  0.7291,  0.1206, -0.0410],
                      [-2.1992, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
